# 機械学習をPythonで実践する-21　　～ ハイパーパラメータのチューニング ～

In [73]:
%load_ext autoreload
%autoreload 2
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder, LabelEncoder, OneHotEncoder
# # import statsmodels.api as sma
from sklearn.model_selection import train_test_split ,cross_val_score, KFold, RepeatedKFold,StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.pipeline import Pipeline
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, log_loss, confusion_matrix,ConfusionMatrixDisplay, \
accuracy_score, precision_score, recall_score,precision_recall_curve,f1_score,roc_curve,auc,get_scorer_names,roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from lightGBM_cv import lightGBM_classifier_cv_func
from category_encoders import TargetEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.datasets import fetch_california_housing

%matplotlib inline
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Grid Search
練習用としてsklearnのcalifornia housingのデータセットを用いる。  
カリフォルニア州の家の値段の中央値を目的変数とするデータセットである。

In [56]:
# データセットを読み込み
data_set = fetch_california_housing()

In [67]:
# データセットの中身
data_set

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [70]:
#  特徴量と目的変数読み込み。yはnumpyのままで問題ない。
X = pl.DataFrame(data_set['data'], schema=data_set['feature_names'])
y = data_set['target']

In [78]:
# 学習データと検証データに分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [79]:
# モデル定義
model = lgb.LGBMRegressor(random_state=0)
# GridSerchに使うパラメタを辞書型で定義
param_grid = {'num_leaves': [10, 20, 30, 40, 50, 60],
              'max_depth': [5, 10, 15],
              'reg_alpha': [0, 0.01, 0.03]}

cv = KFold(n_splits=3, random_state=0, shuffle=True)
# scoringに何も指定しない(default)と、LightGBM回帰の場合はRの2乗値になる。
gs = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv)


In [80]:
# early_stopping
callbacks = [lgb.early_stopping(stopping_rounds=10)]
eval_set = [(X_val, y_val)]
# fit時の引数を定義
fit_params = {'callbacks': callbacks, 'eval_set': eval_set}
# fit_paramsはアンパッキングオペレータ付きの辞書型で渡す。
gs.fit(X_train, y_train, **fit_params)


Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.264841
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.255538
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.255538
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.240402
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.240164
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.240164
Did not meet ea

GridSearchCV(cv=KFold(n_splits=3, random_state=0, shuffle=True),
             estimator=LGBMRegressor(random_state=0),
             param_grid={'max_depth': [5, 10, 15],
                         'num_leaves': [10, 20, 30, 40, 50, 60],
                         'reg_alpha': [0, 0.01, 0.03]})

過程を見ると、early stoppingし切れていないが、  
時間かかるので今回はデフォルトの100回で止める。

In [83]:
cv_results_df = pl.DataFrame(gs.cv_results_) 
cv_results_df

mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_num_leaves,param_reg_alpha,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
f64,f64,f64,f64,object,object,object,struct[3],f64,f64,f64,f64,f64,i32
0.61928,0.618966,0.006718,0.000501,5,10,0,"{5,10,0}",0.797096,0.807785,0.805461,0.803448,0.00459,54
0.140554,0.033333,0.007027,0.000751,5,10,0.01,"{5,10,0}",0.798894,0.806981,0.80605,0.803975,0.003613,53
0.133727,0.021103,0.00836,0.000395,5,10,0.03,"{5,10,0}",0.799277,0.806364,0.806658,0.8041,0.003413,52
0.149566,0.02227,0.007869,0.000818,5,20,0,"{5,20,0}",0.811435,0.821243,0.815893,0.81619,0.00401,36
0.156394,0.0204,0.008246,0.00153,5,20,0.01,"{5,20,0}",0.810028,0.821356,0.813901,0.815095,0.004701,45
0.150316,0.031231,0.009292,0.001245,5,20,0.03,"{5,20,0}",0.812573,0.822185,0.81581,0.816856,0.003993,34
0.170911,0.020553,0.009623,0.000726,5,30,0,"{5,30,0}",0.810847,0.821337,0.816335,0.816173,0.004284,37
0.150881,0.016546,0.007423,0.00016,5,30,0.01,"{5,30,0}",0.811256,0.821676,0.816122,0.816351,0.004257,35
0.15758,0.013528,0.010042,0.001397,5,30,0.03,"{5,30,0}",0.81046,0.821733,0.814879,0.815691,0.004638,41
